In [ ]:
import sys
sys.version

In [ ]:
import os
import pandas as pd
import numpy as np
import glob

from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Normalization, MaxPooling2D, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
# 이미지 폴더에서 파일을 찾아 전처리
class AutoImageClassification:
    def __init__(self, path):
        self.df = self._create_df(path)
        self.train_df, self.test_df = self._split_data(self.df, test_size=0.2)

    @staticmethod
    def _create_df(path):
        files1 = glob.glob(os.path.join(path, "정상", "*.png")) 
        files2 = glob.glob(os.path.join(path, "불량", "*.png"))

        # 파일 개수 출력
        print('정상 사진 개수: ', len(files1))
        print('불량 사진 개수: ', len(files2))
        
        df_n = pd.DataFrame()
        df_p = pd.DataFrame()
        df_n['name'] = [x for x in files2]
        df_n['outcome'] = 0.
        df_p['name'] = [x for x in files1]
        df_p['outcome'] = 1.
        
        # 두 DataFrame 합치고 랜덤 섞기
        df = pd.concat([df_n, df_p], axis=0, ignore_index=True)
        df = shuffle(df)
        return df

    # # stratify 옵션을 주어 outcome 비율(정상/불량)을 유지하면서 train/test 분리
    @staticmethod
    def _split_data(df, test_size=0.2):
        train_df, test_df = train_test_split(df, test_size=test_size, stratify=df['outcome'])
        return train_df, test_df

    # 이미지 파일 전처리
    @staticmethod
    def img_preprocess(path):
        img = Image.open(path)
        # RGB 3채널로 통일 (흑백/투명 배경 대비)
        img = img.convert('RGB')
        img = img.resize((256, 256))
         # numpy 배열로 변환 (모델 학습용 데이터 형식)
        img = np.asarray(img)
        return img

    # DataFrame을 입력받아 X(이미지 데이터), y(레이블) 배열 생성 전처리 후 numpy array 변환
    def create_x_and_y(self, df):
        X = np.array([self.img_preprocess(p) for p in df.name.values]) 
        y = df.outcome.values
        return X, y

dir_path = os.path.join(os.getcwd(), 'image')
aic = AutoImageClassification(path=dir_path)

# 학습 데이터 준비
X_train, y_train = aic.create_x_and_y(aic.train_df)
X_test, y_test = aic.create_x_and_y(aic.test_df)

In [ ]:
# CNN 모델 학습
def create_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Normalization()(inputs)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1)(x)
    outputs = Activation('sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# 모델 생성 및 컴파일
input_shape = (256, 256, 3)
model = create_model(input_shape)
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 구조 출력
model.summary()

# 모델 학습
model.fit(x=X_train, y=y_train, epochs=5, validation_split=0.2, verbose=2)

# 학습된 모델 저장
model.save("model.keras")

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).flatten()

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=[1, 0])
print(cm)

In [7]:
def imageCheck(img, model):

    model = load_model(model)

    # 테스트 이미지를 전처리하는 함수
    def preprocess_image(img_path):
        img = image.load_img(img_path, target_size=(256, 256))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0 
        return img_array

    # 이미지 분석 결과 값을 출력
    def predict_image(img_path):
        img_array = preprocess_image(img_path)
        prediction = model.predict(img_array)
        predicted_class = round(prediction[0][0], 0)
        if predicted_class == 0:
            return "불량"
        else:
            return "정상"

    result = predict_image(img)
    
    return print(f'이미지는 {result}으로 분류되었습니다.')

In [ ]:
imageCheck('불량1.png', 'model.keras')